In [14]:
import pandas
import numpy
from functools import reduce

pandas.options.mode.chained_assignment = None

data = {
    "circ": pandas.read_csv("input/Circulatory Mortality.csv", index_col=False),
    "resp": pandas.read_csv("input/Respiratory Mortality.csv", index_col=False),
    "all": pandas.read_csv("input/All Cause Mortality.csv", index_col=False),
    "circ_est": pandas.read_csv(
        "input/Circulatory Mortality with estimates.csv", index_col=False
    ),
    "resp_est": pandas.read_csv(
        "input/Respiratory Mortality with estimates.csv", index_col=False
    ),
    "all_est": pandas.read_csv(
        "input/All Cause Mortality with estimates.csv", index_col=False
    ),
}


def convert_ids_to_int(df):
    df = df[df["State ANSI"].notnull()]
    df["STATEFP"] = pandas.to_numeric(df["State ANSI"], downcast="integer")
    df["COUNTYFP"] = pandas.to_numeric(df["County ANSI"], downcast="integer")
    return df.drop(columns=["State ANSI", "County ANSI"])


def calculate_mortality(df):
    """
    We calculate mortality in terms of deaths per 1,000 people.
    It's easier to read than deaths per person (because that's such a small number).
    """

    if (
        "Percent Deaths 0-5" in df
        and "Percent Deaths 5-25" in df
        and "Percent Deaths 25+" in df
    ):
        df = df.rename(
            columns={
                "Percent Deaths 0-5": "mort_0-5",
                "Percent Deaths 5-25": "mort_5-25",
                "Percent Deaths 25+": "mort_25+",
            }
        )
        df["mort_0-5"] *= 1000
        df["mort_5-25"] *= 1000
        df["mort_25+"] *= 1000
    else:
        df["mort_0-5"] = (
            df["deaths_0-5"] * 1_000 / df["Population_0-5"].replace({0: numpy.nan})
        )
        df["mort_5-25"] = (
            df["deaths_5-25"] * 1_000 / df["Population_5-25"].replace({0: numpy.nan})
        )
        df["mort_25+"] = (
            df["deaths_25+"] * 1_000 / df["Population_25+"].replace({0: numpy.nan})
        )

    return df


def append_suffix(df, suffix):
    return df.rename(
        columns={
            "deaths_0-5": "deaths_0-5_" + suffix,
            "deaths_25+": "deaths_25+_" + suffix,
            "deaths_5-25": "deaths_5-25_" + suffix,
            "mort_0-5": "mort_0-5_" + suffix,
            "mort_5-25": "mort_5-25_" + suffix,
            "mort_25+": "mort_25+_" + suffix,
        }
    )


rename_map = {
    "Deaths_0-5": "deaths_0-5",
    "Deaths_25+": "deaths_25+",
    "Deaths_5-25": "deaths_5-25",
}


def clean(df, key):
    df = convert_ids_to_int(df)
    df = df.rename(columns=rename_map)
    df = calculate_mortality(df)
    df = append_suffix(df, key)
    df = df.replace({0: numpy.nan})
    df = df.round(6)
    return df


data = {key: clean(df, key) for key, df in data.items()}

data_merged = reduce(
    lambda left, right: pandas.merge(
        left,
        right,
        on=[
            "STATEFP",
            "COUNTYFP",
            "Population_0-5",
            "Population_25+",
            "Population_5-25",
        ],
    ),
    data.values(),
)

data_merged.to_csv("output/mortality.csv", index=None)
data_merged


,deaths_0-5_circ,deaths_25+_circ,deaths_5-25_circ,Population_0-5,Population_25+,Population_5-25,STATEFP,COUNTYFP,mort_0-5_circ,mort_5-25_circ,...,deaths_5-25_resp_est,mort_0-5_resp_est,mort_5-25_resp_est,mort_25+_resp_est,deaths_0-5_all_est,deaths_25+_all_est,deaths_5-25_all_est,mort_0-5_all_est,mort_5-25_all_est,mort_25+_all_est
0,NaN,157.0,NaN,3191.0,37332.0,14893.0,1,1,NaN,NaN,...,NaN,5.468900e-02,0.645634,1.232187e+00,NaN,510.0,NaN,2.366505,0.633185,13.661202
1,NaN,631.0,NaN,11660.0,147731.0,49172.0,1,3,NaN,NaN,...,NaN,5.468900e-02,0.645634,1.171047e+00,11.0,1933.0,23.0,0.943396,0.467746,13.084593
2,NaN,81.0,NaN,1352.0,18326.0,6287.0,1,5,NaN,NaN,...,NaN,5.468900e-02,0.645634,2.042980e+10,NaN,235.0,NaN,2.366505,0.633185,12.823311
3,NaN,73.0,NaN,1274.0,16121.0,5248.0,1,7,NaN,NaN,...,NaN,5.468900e-02,0.645634,8.064020e-01,NaN,224.0,NaN,2.366505,0.633185,13.894920
4,NaN,184.0,NaN,3353.0,39867.0,14484.0,1,9,NaN,NaN,...,NaN,5.468900e-02,0.645634,2.583590e+00,NaN,675.0,NaN,2.366505,0.633185,16.931297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,NaN,83.0,NaN,3107.0,28510.0,12548.0,56,37,NaN,NaN,...,NaN,3.970145e+09,1.603140,4.209050e-01,NaN,298.0,NaN,1.161331,1.288924,10.452473
3136,NaN,NaN,NaN,1209.0,17488.0,4494.0,56,39,NaN,NaN,...,NaN,3.970145e+09,1.603140,2.042980e+10,NaN,74.0,NaN,1.161331,1.288924,4.231473
3137,NaN,25.0,NaN,1634.0,13119.0,6020.0,56,41,NaN,NaN,...,NaN,3.970145e+09,1.603140,2.042980e+10,NaN,137.0,NaN,1.161331,1.288924,10.442869
3138,NaN,NaN,NaN,474.0,5722.0,2039.0,56,43,NaN,NaN,...,NaN,3.970145e+09,1.603140,2.042980e+10,NaN,78.0,NaN,1.161331,1.288924,13.631597
